# Bridging cluster analysis

In [559]:
%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
import anndata
import seaborn as sns

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list(name='gene_cmap', colors=['lightgrey', 'thistle', 'red', 'darkred']) 

sc.settings.set_figure_params(dpi=80, color_map='viridis', vector_friendly=False,  dpi_save=300)

scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.23.4 scipy==1.9.3 pandas==1.5.3 scikit-learn==1.3.2 statsmodels==0.14.0 python-igraph==0.10.4 pynndescent==0.5.8


# Further subcluster

## EtoT

In [ ]:
adata = sc.read('./write/All_outer_leidenv2.h5ad')

In [ ]:
sc.tl.leiden(adata,restrict_to=('Leiden_v2',['16']),resolution=0.4,
             key_added='Leiden_sub_c16')

In [ ]:
c16_5 = adata[adata.obs.Leiden_sub_c16=='16,5']

In [ ]:
sc.pl.umap(c16_5,color='Leiden_sub_c16_5',size=30,
          legend_loc='on data',legend_fontsize='xx-small')

In [ ]:
c16_5.obs['EtoT'] = c16_5.obs.Leiden_sub_c16_5.cat.rename_categories(
    {'16,5,0':'EtoT-3',
     '16,5,1':'EtoT-6',
     '16,5,2':'EtoT-5',
     '16,5,3':'EtoT-7',
     '16,5,4':'EtoT-2',
     '16,5,5':'EtoT-4',
     '16,5,6':'EtoT-1'}
     )

In [ ]:
c16_5.obs['EtoT'] = c16_5.obs['EtoT'].cat.reorder_categories(['EtoT-1','EtoT-2','EtoT-3','EtoT-4','EtoT-5','EtoT-6','EtoT-7'])

In [ ]:
sc.pl.umap(c16_5,color='EtoT',size=30)

## EtoM

In [ ]:
sc.tl.leiden(adata,restrict_to=('Leiden_v2',['27']),resolution=0.4,
             key_added='Leiden_sub_c27')

In [ ]:
c27 = adata[adata.obs.Leiden_v2=='27']

In [ ]:

sc.pl.umap(c27,color='Leiden_sub_c27',size=30,
          #legend_loc='on data',legend_fontsize='xx-small'
          )

In [ ]:
c27.obs['EtoM'] = c27.obs.Leiden_sub_c27.cat.rename_categories(
    {'27,0':'EtoM-3',
     '27,1':'EtoM-4',
     '27,2':'EtoM-2',
     '27,3':'EtoM-1',
     '27,4':'EtoM-5',
     '27,5':'EtoM-6'}
     )

In [ ]:
c27.obs['EtoM'] = c27.obs['EtoM'].cat.reorder_categories(['EtoM-1','EtoM-2','EtoM-3','EtoM-4','EtoM-5','EtoM-6'])

In [ ]:
sc.pl.umap(c27,color='EtoM',size=30)

## TtoM

In [ ]:
sc.tl.leiden(adata,restrict_to=['Leiden_v2',['20']],key_added='Leiden_sub_c20',resolution=0.3)

In [ ]:
sc.pl.umap(adata,color='Leiden_sub_c20',groups=['20,4'],size=5,palette=['#1f77b4'])

In [ ]:
c20 = adata[adata.obs.Leiden_v2=='20']

In [ ]:
sc.tl.leiden(c20,restrict_to=['Leiden_sub_c20',['20,4']],key_added='Leiden_sub_c20_4',resolution=0.8)

In [ ]:
sc.pl.umap(c20,color='Leiden_sub_c20_4',size=5)

In [ ]:
c20_4 = c20[c20.obs.Leiden_sub_c20=='20,4']

In [ ]:
sc.pl.umap(c20_4,color='Leiden_sub_c20_4',size=50,
          #legend_loc='on data',legend_fontsize='xx-small'
          )

In [ ]:
c20_4.obs['TtoM'] = c20_4.obs.Leiden_sub_c20_4.cat.rename_categories(
    {'20,4,0':'TtoM-2',
     '20,4,1':'TtoM-1',
     '20,4,2':'TtoM-3',}
     )

In [ ]:
c20_4.obs['TtoM'] = c20_4.obs['TtoM'].cat.reorder_categories(['TtoM-1','TtoM-2','TtoM-3'])

In [ ]:
sc.pl.umap(c20_5,color='TtoM',size=50)

# Combined

In [ ]:
c27.obs['Leiden_sub'] = 'c27_EtoM'
c16_5.obs['Leiden_sub'] = 'c16_5_EtoT'
c20_4.obs['Leiden_sub'] = 'c20_4_TtoM'

In [ ]:
adata_bridge = anndata.concat([c27,c20_4,c16_5])

# Dotplots

In [ ]:
GL = ['kdrl','cdh5','notch1b','gata1a','gata2a','gata2b','itga2b','tal1','myb','spi1b']

In [ ]:
Todraw = 'Thrombo_To_Macro'
TD = c16_5 if Todraw=='Endo_To_Thrombo' else (c27 if Todraw=='Endo_To_Macro' else c20_4)
groupby = 'EtoT' if Todraw=='Endo_To_Thrombo' else ('EtoM' if Todraw=='Endo_To_Macro' else 'TtoM')
sc.pl.dotplot(TD,var_names=GL,groupby=groupby,swap_axes=True,standard_scale='var',
              title=Todraw,save=f'S2_{Todraw}.genes.pdf'
             )

# UMAPs

In [ ]:
sc.pl.umap(c27,color=['phase','db_scores'],size=30,
           wspace=0.2,
           cmap=cmap,
           save='_c17_basic2.png')

In [ ]:
adata = sc.read('./write/All_outer_Leidenv2.h5ad') # All_cells

In [ ]:
adata.obs['Leiden_sub'] = adata_bridge.obs['Leiden_sub']

In [ ]:
sc.pl.umap(adata,color='Leiden_sub',size=5,
          save='_3_bridges.pdf')

sc.pl.umap(adata,color='Leiden_v2',groups=['16','20','27'],size=5,
          save'_3_bridge_clus.pdf)

## Proportion analysis

In [ ]:
# c27,c16,c25, c20, WT_filtered, COMBINED
adata2.obs['Con'] = adata2.obs['Day'].astype(str)+'_'+adata2.obs['Author'].astype(str) 
adata2.obs['Leiden_v2'] = adata[np.intersect1d(adata.obs_names,adata2.obs_names)].obs['Leiden_v2']
cl = ['16', '20', '25','27']
tab1 = pd.crosstab(adata2[np.in1d(adata2.obs['Leiden_v2'], cl)].obs['Leiden_v2'], adata2.obs['Con'])

In [ ]:
print(pd.crosstab(adata2.obs.Day,adata2.obs.Condition))

In [ ]:
total_cnt = adata2.obs['Con'].value_counts()

In [ ]:
tab2 = tab1.T.join(total_cnt)

In [ ]:
tab2.head()
tab2.to_csv('./Files/tab2_WT_filtered_Combined.csv')

In [ ]:
# c27,c16_5 and c20_4. All cells
All_cells.obs['Con'] = All_cells.obs['Day'].astype(str)+'_'+All_cells.obs['Author'].astype(str)
bridge.obs['Con'] = bridge.obs['Day'].astype(str)+'_'+bridge.obs['Author'].astype(str) 
cl = ['c16_5_EtoT', 'c20_4_TtoM', 'c27_EtoM']
tab1 = pd.crosstab(bridge[np.in1d(bridge.obs['Leiden_sub'], cl)].obs['Leiden_sub'], bridge.obs['Con'])

In [ ]:
total_cnt = adata2.obs['Con'].value_counts()

In [ ]:
tab2 = tab1.T.join(total_cnt)

In [ ]:
print(tab2.head())

In [ ]:
tab2.to_csv('./Files/tab2_bridge_Combined.csv')